In [1]:
import numpy as np
import pandas as pd

In [2]:
from env import *

In [3]:
def preprocess(tickpath, filename):
    tickdf = pd.read_csv(os.path.join(tickpath, filename),names=['date','time','ms','lastprice','volume','bid','bidv','ask','askv','opi','tur','contract'])
    tickdf['datestr'] = tickdf['date'].apply(lambda x: '%s-%s-%s'%(str(x)[:4], str(x)[4:6], str(x)[6:8]))
    tickdf['timestr'] = tickdf['time'].apply(lambda x: '%s:%s:%s'%(format(x, '06d')[:-4], format(x, '06d')[-4:-2], format(x, '06d')[-2:]))
    tickdf['timestr'] = tickdf['timestr']+'.'+tickdf['ms'].apply(lambda x: format(x, '03d'))
    tickdf = tickdf.set_index(pd.to_datetime(tickdf['datestr']+' '+tickdf['timestr']))
    tickdf.index.name = 'datetime'
    tickdf.loc[tickdf['bid']<0.9*tickdf['lastprice'], 'bid'] = np.nan
    tickdf.loc[tickdf['ask']<0.9*tickdf['lastprice'], 'ask'] = np.nan
    tickdf[['bid','ask']] = tickdf[['bid','ask']].fillna(method='ffill').fillna(tickdf.iloc[0]['lastprice'])
    tickdf['midprice'] = (tickdf['ask']+tickdf['bid'])/2.0

    tdata = tickdf[['bid','ask','bidv','askv','volume']]
    std = tdata.iloc[0,0]
    tdata.loc[:,'bid'] = tdata['bid']-std
    tdata.loc[:,'ask'] = tdata['ask']-std

    def norm(x):
        return (x-x.mean())/x.std()

    tdata.loc[:,'askv'] = norm(tdata.askv)
    tdata.loc[:,'bidv'] = norm(tdata.bidv)
    tdata.loc[:,'volume'] = norm(tdata.volume)
    return tdata

In [4]:
import os
cwd = '/data/data_zltick/rb'
datas = []
for _,__, files in os.walk(cwd):
    for file in files:
        if int(file[:4])<2021:
            continue
        data = preprocess(cwd, file)
        datas.append(data)

/home/hongyuan/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [5]:
# import matplotlib.pyplot as plt
# fig, ax1 = plt.subplots()

# ax2 = ax1.twinx()
# ax1.plot(tdata.bid[500:800])
# ax2.plot(tdata.volume[500:800], color='red')

# plt.show()

In [6]:
env = BaseMarket(datas=datas,back_length=100,time_limit=600)

In [12]:
tot = 0
env.seed(567)
for i in range(1024):
    done = False
    state = env.reset()
    action = 0
    while not done:
        state, reward, done, _ = env.step(action)
        if state[0] == 1:
            action = 1
    tot += reward
print(tot/1024)

0.06640625


In [13]:
env.seed(567)
tot = 0
for i in range(1024):
    done = False
    state = env.reset()
    while not done:
        action = env.action_space.sample()
        state, reward, done, _ = env.step(action)
    tot += reward
print(tot/1024)

-0.0439453125


In [9]:
import elegantrl.agent as agent
from elegantrl.config import Arguments
env_args = {
    'env_num': 1,
    'env_name': 'executioner',
    'max_step': 600,
    'state_dim': (100+1)*5,
    'action_dim': 2,
    'if_discrete': True,
    'target_return': 0.4
}

In [10]:
args = Arguments(agent.AgentD3QN,env=env, env_args=env_args)

In [11]:
from elegantrl.run import train_and_evaluate

In [16]:
args.target_step = args.max_step * 50
args.eval_times = 2 ** 10
args.net_dim = 512
args.batch_size = 512
args.gamma = 1
args.explore_rate = 0.1
args.seed = 567
args.learning_rate = 2**-15
train_and_evaluate(args)

| Arguments Remove cwd: ./executioner_D3QN_0
################################################################################
ID     Step    maxR |    avgR   stdR   avgS  stdS |    expR   objC   etc.
0  3.00e+04    0.06 |
0  3.00e+04    0.06 |    0.06    3.4     90   153 |   -0.05   0.08  -0.06
0  1.20e+05    0.06 |    0.03    3.2     77   133 |   -0.00   0.07  -0.06
0  2.40e+05    0.06 |
0  2.40e+05    0.06 |    0.06    1.2     13    28 |    0.01   0.10  -0.53
0  3.60e+05    0.08 |
0  3.60e+05    0.08 |    0.08    0.9      8    21 |   -0.04   0.11  -0.43
0  4.80e+05    0.15 |
0  4.80e+05    0.15 |    0.15    2.5     65   139 |    0.00   0.10  -0.24
0  5.70e+05    0.19 |
0  5.70e+05    0.19 |    0.19    1.9     32    79 |    0.02   0.08  -0.26
0  6.60e+05    0.22 |
0  6.60e+05    0.22 |    0.22    0.9      9    17 |    0.03   0.07  -0.28
0  7.80e+05    0.23 |
0  7.80e+05    0.23 |    0.23    0.9      9    21 |    0.04   0.11  -0.26
0  8.70e+05    0.23 |    0.22    1.0     12    24 |   

In [17]:
# age = args.agent().save_or_load_agent(cwd='./executioner_D3QN_0', if_save=False)
from elegantrl.run import *

In [18]:
import torch
torch.set_grad_enabled(False)
# args.init_before_training()
gpu_id = args.learner_gpus

'''init'''
env = build_env(args.env, args.env_func, args.env_args)

age = init_agent(args, gpu_id, env)

In [19]:
age.save_or_load_agent('./executioner_D3QN_0', if_save=False)

In [20]:
# for seed in range(100):
seed = 4353
env.seed(seed)
tot = 0
for i in range(1000):
    done = False
    state = env.reset()
    steps = 0
    # print(state.reshape((-1, 5)))
    while not done:
        steps += 1
        ten_s = torch.as_tensor(state, dtype=torch.float32).unsqueeze(0)
        ten_a = age.act(ten_s)
        # print(ten_a)
        ten_a = ten_a.argmax(dim=1)
        state, reward, done, _ = env.step(ten_a[0].numpy())  # different
        # print(state[-5:])
        # print(reward)
    tot += reward
    # env.episode_data[:300].to_csv('tmp.csv')
    # break
    # print(steps)
print(tot/1000)

0.348
0.301
0.301
0.163
0.316
0.346
0.231
0.284
0.257
0.312
0.255
0.322
0.317
0.331
0.257
0.168
0.332
0.293
0.375
0.272
0.306
0.317
0.15
0.285
0.347
0.334
0.353
0.259
0.177
0.265
0.366
0.316
0.234
0.187
0.277
0.294
0.283
0.376
0.257
0.353
0.3
0.337
0.318
0.341
0.318
0.239
0.291
0.307
0.26
0.377
0.367
0.356
0.239
0.341
0.244
0.28
0.314
0.235
0.299
0.373
0.162
0.439
0.301
0.292
0.37
0.296
0.3
0.329
0.298
0.38
0.328
0.325
0.356
0.366
0.219
0.268
0.364
0.228
0.305
0.389
0.33
0.277
0.341
0.34
0.288
0.297
0.29
0.201
0.318
0.283
0.328
0.368
0.357


KeyboardInterrupt: 

In [21]:
for seed in range(100):
    tot = 0
    env.seed(seed)
    for i in range(1000):
        done = False
        state = env.reset()
        action = 0
        while not done:
            state, reward, done, _ = env.step(action)
            if state[0] == 1:
                action = 1
        tot += reward
    print(tot/1000)

0.193
-0.09
-0.106


KeyboardInterrupt: 

In [38]:
tot = 0
env.seed(1)
for i in range(1000):
    done = False
    state = env.reset()
    action = 1
    while not done:
        state, reward, done, _ = env.step(action)
    tot += reward
print(tot/1000)

-0.046


In [41]:
tot = 0
env.seed(1)
for i in range(10000):
    done = False
    state = env.reset()
    while not done:
        action = env.action_space.sample()
        state, reward, done, _ = env.step(action)
    tot += reward
print(tot/10000)

-0.0302


In [28]:
tmp = np.load(r'executioner_D3QN_0\replay_0.npz')

In [40]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('tkagg')
plt.plot(data['bid'])
plt.show()

In [35]:
tmp['buf_other']

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       ...,
       [ 0.  ,  0.99,  2.  ],
       [ 0.  ,  0.99,  2.  ],
       [-0.  ,  0.  ,  2.  ]], dtype=float16)

In [5]:
a = [(1,2,3),(3,4,5),(5,6,5),(7,8,5)]
b = list(map(list, zip(*a)))

In [7]:
import torch
b

[[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 5, 5]]